In [1]:
"""
Based from : https://github.com/sjchoi86/tensorflow-101/blob/master/notebooks/cnn_mnist_simple.ipynb
"""

import tensorflow as tf
import os
import sys
import datetime
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Activation
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
if not os.path.exists('../dataset/mnist'):
    os.mkdir('../dataset/mnist')
   
mnist = input_data.read_data_sets('../dataset/mnist', one_hot=True)
trainimg    = mnist.train.images
trainlabel  = mnist.train.labels
testimg     = mnist.test.images
testlabel   = mnist.test.labels

Extracting ../dataset/mnist/train-images-idx3-ubyte.gz
Extracting ../dataset/mnist/train-labels-idx1-ubyte.gz
Extracting ../dataset/mnist/t10k-images-idx3-ubyte.gz
Extracting ../dataset/mnist/t10k-labels-idx1-ubyte.gz


In [7]:
learning_rate = 0.001
training_epochs = 2
batch_size = 100
display_step = 1

In [4]:
def do_train(device):
    if device == 'gpu':
        device_type = '/gpu:0'
    else:
        device_type = '/cpu:0'
        
    with tf.device(device_type): # <= This is optional
        n_input  = 784
        n_output = 10
        weights  = {
            'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64], stddev=0.1)),
            'wd1': tf.Variable(tf.random_normal([14*14*64, n_output], stddev=0.1))
        }
        biases   = {
            'bc1': tf.Variable(tf.random_normal([64], stddev=0.1)),
            'bd1': tf.Variable(tf.random_normal([n_output], stddev=0.1))
        }
        def conv_simple(_input, _w, _b):
            # Reshape input
            _input_r = tf.reshape(_input, shape=[-1, 28, 28, 1])
            # Convolution
            _conv1 = tf.nn.conv2d(_input_r, _w['wc1'], strides=[1, 1, 1, 1], padding='SAME')
            # Add-bias
            _conv2 = tf.nn.bias_add(_conv1, _b['bc1'])
            # Pass ReLu
            _conv3 = tf.nn.relu(_conv2)
            # Max-pooling
            _pool  = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            # Vectorize
            _dense = tf.reshape(_pool, [-1, _w['wd1'].get_shape().as_list()[0]])
            # Fully-connected layer
            _out = tf.add(tf.matmul(_dense, _w['wd1']), _b['bd1'])
            # Return everything
            out = {
                'input_r': _input_r, 'conv1': _conv1, 'conv2': _conv2, 'conv3': _conv3
                , 'pool': _pool, 'dense': _dense, 'out': _out
            }
            return out

        def conv_keras(_input):
            x = Dense(128, activation='relu')(_input)
            x = Dense(128, activation='relu')(x)
            preds = Dense(10, activation='softmax')(x)
            return preds

    print ("CNN ready with {}".format(device_type))

    x = tf.placeholder(tf.float32, [None, n_input])
    y = tf.placeholder(tf.float32, [None, n_output])
   
    with tf.device(device_type):
        _pred = conv_simple(x, weights, biases)['out']
#         preds = conv_keras(x)
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
#         cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(preds, y))
        optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
        _corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
        accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
        init = tf.initialize_all_variables()
    #Saver
#     save_step = 1;
#     savedir = "nets/"
#     saver = tf.train.Saver(max_to_keep=3) 
#     if not os.path.exists('nets'):
#         os.mkdir('nets')
    print ("Network Ready to Go!")
    
    do_train = 1
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
    sess.run(init)
    
    start_time = datetime.datetime.now()
    if do_train == 1:
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                # Fit training using batch data
                sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
                # Compute average loss
                avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch

            # Display logs per epoch step
            if epoch % display_step == 0: 
                print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
                train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
                print (" Training accuracy: %.3f" % (train_acc))
                test_acc = sess.run(accr, feed_dict={x: testimg, y: testlabel})
                print (" Test accuracy: %.3f" % (test_acc))

            # Save Net
#             if epoch % save_step == 0:
#                 saver.save(sess, "nets/cnn_mnist_simple.ckpt-" + str(epoch))
        print ("Optimization Finished.")
        print ("Single {} computaion time : {}".format(device, datetime.datetime.now() - start_time))

In [8]:
do_train('gpu')

CNN ready with /gpu:0
Network Ready to Go!
Epoch: 000/100 cost: 1.582141847
 Training accuracy: 0.100
 Test accuracy: 0.090
Epoch: 001/100 cost: 1.514721184
 Training accuracy: 0.090
 Test accuracy: 0.090
Epoch: 002/100 cost: 1.501261046
 Training accuracy: 0.090
 Test accuracy: 0.090
Epoch: 003/100 cost: 1.493142263
 Training accuracy: 0.060
 Test accuracy: 0.090
Epoch: 004/100 cost: 1.488540890
 Training accuracy: 0.060
 Test accuracy: 0.090
Epoch: 005/100 cost: 1.483906736
 Training accuracy: 0.090
 Test accuracy: 0.090
Epoch: 006/100 cost: 1.481189398
 Training accuracy: 0.080
 Test accuracy: 0.090
Epoch: 007/100 cost: 1.478892675
 Training accuracy: 0.060
 Test accuracy: 0.090
Epoch: 008/100 cost: 1.477367144
 Training accuracy: 0.050
 Test accuracy: 0.090
Epoch: 009/100 cost: 1.475797993
 Training accuracy: 0.070
 Test accuracy: 0.090
Epoch: 010/100 cost: 1.475287900
 Training accuracy: 0.070
 Test accuracy: 0.090
Epoch: 011/100 cost: 1.473907072
 Training accuracy: 0.100
 Test a

In [9]:
do_train('cpu')

CNN ready with /cpu:0
Network Ready to Go!
Epoch: 000/100 cost: 1.584374230
 Training accuracy: 0.090
 Test accuracy: 0.123
Epoch: 001/100 cost: 1.513501439
 Training accuracy: 0.130
 Test accuracy: 0.123
Epoch: 002/100 cost: 1.500698281
 Training accuracy: 0.110
 Test accuracy: 0.123
Epoch: 003/100 cost: 1.493261100
 Training accuracy: 0.150
 Test accuracy: 0.123
Epoch: 004/100 cost: 1.487881976
 Training accuracy: 0.130
 Test accuracy: 0.123
Epoch: 005/100 cost: 1.484164736
 Training accuracy: 0.150
 Test accuracy: 0.123
Epoch: 006/100 cost: 1.481189549
 Training accuracy: 0.150
 Test accuracy: 0.123
Epoch: 007/100 cost: 1.479455519
 Training accuracy: 0.080
 Test accuracy: 0.123
Epoch: 008/100 cost: 1.477848136
 Training accuracy: 0.080
 Test accuracy: 0.123
Epoch: 009/100 cost: 1.476350325
 Training accuracy: 0.150
 Test accuracy: 0.123
Epoch: 010/100 cost: 1.475232162
 Training accuracy: 0.090
 Test accuracy: 0.123
Epoch: 011/100 cost: 1.474210403
 Training accuracy: 0.100
 Test a